In [1]:
import censusdata
import pandas as pd

In [2]:
#function to make list of all county ids in state (given by census state id)
def county_list(state_number):
    counties = censusdata.geographies(censusdata.censusgeo([('state', state_number), ('county','*')]), 'acs5', 2018)
    county_list = []
    for i in counties.keys():
        county_list.append(counties[i].geo[1][1])
    return county_list

#function to pull defined variables for blocks in specified state, looping over counties (can't pull all blocks in a state)
#(input state id and list of variables)
def block_pull(state_id, variable_list):
    c_list = county_list(state_id)
    for i in range(0, len(c_list)):
        geo = censusdata.censusgeo([('state', state_id), ('county', c_list[i]), ('tract','*'),('block group','*')])
        county_df = censusdata.download('acs5', 2018, geo, variable_list)
        if i == 0:
            data = county_df
        else:
            data = pd.concat([data, county_df])
    return data

In [3]:
#just one variable in here for now, need to select and add variables
variables_list = ['B02001_001E']

block_data = block_pull('39', variables_list)

In [12]:
# SAVE DATA, INDEX IS JUST FOR TESTING
data = block_data
data['index']=range(len(data))
data.head()

,B02001_001E,index
"Block Group 3, Census Tract 9540, Gallia County, Ohio: Summary level: 150, state:39> county:053> tract:954000> block group:3",1348,0
"Block Group 2, Census Tract 9540, Gallia County, Ohio: Summary level: 150, state:39> county:053> tract:954000> block group:2",621,1
"Block Group 1, Census Tract 9540, Gallia County, Ohio: Summary level: 150, state:39> county:053> tract:954000> block group:1",1125,2
"Block Group 2, Census Tract 9539, Gallia County, Ohio: Summary level: 150, state:39> county:053> tract:953900> block group:2",1640,3
"Block Group 1, Census Tract 9539, Gallia County, Ohio: Summary level: 150, state:39> county:053> tract:953900> block group:1",906,4


In [13]:
data.to_csv('test.csv', index=False)